# MNIST - Convolution Neural Network
## More deep, Visdom

In [1]:
import torch
import torch.nn as nn
from torchsummary import summary
import torchvision.datasets as dsets
import torchvision.transforms as transforms

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [4]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)
mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [5]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [6]:
# CNN Model (2 conv layers)
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc1 = nn.Linear(3*3*128, 625, bias=True)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)  # Flatten them for FC
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [7]:
# instantiate CNN model
model = CNN().to(device) 

In [8]:
summary(model, (1, 28, 28)) 

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
              ReLU-2           [-1, 32, 28, 28]               0
         MaxPool2d-3           [-1, 32, 14, 14]               0
            Conv2d-4           [-1, 64, 14, 14]          18,496
              ReLU-5           [-1, 64, 14, 14]               0
         MaxPool2d-6             [-1, 64, 7, 7]               0
            Conv2d-7            [-1, 128, 7, 7]          73,856
              ReLU-8            [-1, 128, 7, 7]               0
         MaxPool2d-9            [-1, 128, 3, 3]               0
           Linear-10                  [-1, 625]         720,625
             ReLU-11                  [-1, 625]               0
           Linear-12                   [-1, 10]           6,260
Total params: 819,557
Trainable params: 819,557
Non-trainable params: 0
-------------------------------

In [9]:
# define cost/loss & optimizer
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

### Make plot

In [10]:
import visdom

vis = visdom.Visdom()
vis.close(env='main') 

Setting up a new session...


''

In [11]:
def loss_tracker(loss_plot, loss_value, num):
    vis.line(X=num, Y=loss_value,
             win=loss_plot, update='append')

In [12]:
loss_plt = vis.line(Y=torch.Tensor(1).zero_(), 
                    opts=dict(title='loss_tracker', legend=['loss'], showlegend=True)) 

### Train with loss_tracker

In [13]:
# Training
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('[Epoch: {:>2}] cost = {:>.9}'.format(epoch + 1, avg_cost))
    loss_tracker(loss_plt, torch.Tensor([avg_cost]), torch.Tensor([epoch]))
    
print('Learning Finished!')

[Epoch:  1] cost = 0.159733817
[Epoch:  2] cost = 0.0429320447
[Epoch:  3] cost = 0.0300062932
[Epoch:  4] cost = 0.0221342947
[Epoch:  5] cost = 0.0190195721
[Epoch:  6] cost = 0.0138610471
[Epoch:  7] cost = 0.0127287945
[Epoch:  8] cost = 0.010186593
[Epoch:  9] cost = 0.010345838
[Epoch: 10] cost = 0.00772080291
[Epoch: 11] cost = 0.00958343316
[Epoch: 12] cost = 0.00630227057
[Epoch: 13] cost = 0.00776766613
[Epoch: 14] cost = 0.00484489603
[Epoch: 15] cost = 0.00470292382
Learning Finished!


![image](images/32_MNIST_Loss_Tracker.svg)

In [14]:
test_loader = torch.utils.data.DataLoader(dataset=mnist_test,
                                          batch_size=batch_size,
                                          shuffle=False,
                                          drop_last=False)
test_batch = len(test_loader)

In [15]:
# Test model and check accuracy
with torch.no_grad():
    mean_acc = 0.
    for X_test, Y_test in test_loader:
        X_test = X_test.to(device)
        Y_test = Y_test.to(device)
    
        prediction = model(X_test)
        correct_prediction = torch.argmax(prediction, 1) == Y_test
        accuracy = correct_prediction.float().mean()
        
        mean_acc += accuracy.item() / test_batch
        
    print('Accuracy:', mean_acc)   

Accuracy: 0.9904999774694448
